In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
stemmer=PorterStemmer()
data_frame = pd.read_csv("train_data.csv")


df=data_frame.rename(columns={'i didnt feel humiliated':'text','sadness':'emotion'})
df.dropna()


,text,emotion
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness
...,...,...
15994,i just had a very brief time in the beanbag an...,sadness
15995,i am now turning and i feel pathetic that i am...,sadness
15996,i feel strong and good overall,joy
15997,i feel like this was such a rude comment and i...,anger


In [3]:
df.drop_duplicates(subset ="text", 
                     keep = False, inplace = True) 
df.replace("[^a-zA-Z]"," ",regex=True,inplace=True)
ind=df.columns

In [4]:
df=df.reset_index()

In [5]:
df=df.drop(['index'],axis=1)
Y = pd.get_dummies(df['emotion']).values

In [6]:
for indexx in ind:
    df[indexx]=df[indexx].str.lower()

In [7]:
text=df['text']
corpus=[]

In [8]:
for i in text:
    words=nltk.word_tokenize(i)
    words=[stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
    i=' '.join(words)
    corpus.append(i)


In [9]:
import tensorflow

In [10]:
from tensorflow.keras.layers import Embedding , LSTM
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
voc_size=10000

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
onehot = [one_hot(words,voc_size) for words in corpus]


In [13]:
sent_length=30

In [14]:
embedded_docs= pad_sequences(onehot,padding='pre',maxlen=sent_length)

In [15]:
x_train=np.array(embedded_docs)
y_train=np.array(Y)

In [16]:
embedding_vector_length = 32 

In [17]:
from tensorflow.keras.layers import Dropout,Dense

In [18]:
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_length, input_length=x_train.shape[1]))

model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 15
batch_size = 64

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/15
250/250 [==============================] - 32s 88ms/step - loss: 1.5815 - accuracy: 0.3621
Epoch 2/15
250/250 [==============================] - 27s 106ms/step - loss: 0.8043 - accuracy: 0.7378
Epoch 3/15
250/250 [==============================] - 35s 139ms/step - loss: 0.3832 - accuracy: 0.8682
Epoch 4/15
250/250 [==============================] - 19s 76ms/step - loss: 0.2426 - accuracy: 0.9152
Epoch 5/15
250/250 [==============================] - 21s 86ms/step - loss: 0.1998 - accuracy: 0.9314
Epoch 6/15
250/250 [==============================] - 21s 84ms/step - loss: 0.1541 - accuracy: 0.9447
Epoch 7/15
250/250 [==============================] - 24s 94ms/step - loss: 0.1268 - accuracy: 0.9532
Epoch 8/15
250/250 [==============================] - 23s 94ms/step - loss: 0.1063 - accuracy: 0.9595
Epoch 9/15
250/250 [==============================] - 22s 89ms/step - loss: 0.0993 - accuracy: 0.9651
Epoch 10/15
250/250 [==============================] - 24s 95ms/step - loss: 0.0

In [19]:
test_frame=pd.read_csv("test_data.csv")

test_df=test_frame.rename(columns={'im feeling rather rotten so im not very ambitious right now':'text','sadness':'emotion'})

test_df.dropna( )

test_df.drop_duplicates(subset ="text", 
                     keep = False, inplace = True) 

test_df.replace("[^a-zA-Z]"," ",regex=True,inplace=True)

test_ind=test_df.columns

test_df=test_df.reset_index()

In [20]:
test_df=test_df.drop(['index'],axis=1)
test_Y=pd.get_dummies(test_df['emotion']).values

In [21]:
for indexx in test_ind:
    test_df[indexx]=test_df[indexx].str.lower()

In [22]:
test_text=test_df['text']
test_corpus=[]

In [23]:
for i in test_text:
    words=nltk.word_tokenize(i)
    words=[stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
    i=' '.join(words)
    test_corpus.append(i)
    

In [24]:
test_onehot=[one_hot(words,voc_size) for words in test_corpus]
test_embedded_docs=pad_sequences(test_onehot,padding='pre',maxlen=sent_length)
x_test=np.array(test_embedded_docs)
y_test=np.array(test_Y)

In [25]:
predictions=model.predict(x_test)

In [26]:
final_y=[]    
for i in range(len(y_test)):
    l=y_test[i].tolist()
    final_y.append(l.index(max(l))) 
                   

In [27]:
final_result=[]    
for i in range(len(predictions)):
    l=predictions[i].tolist()
    final_result.append(l.index(max(l))) 

In [28]:
from sklearn.metrics import accuracy_score
score=accuracy_score(final_y,final_result)
print(score)

0.8534267133566783
